# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Namalenga,-10.95,39.10,70.52,88,19,3.02,TZ,1604007593
1,1,Gao,16.64,1.64,82.51,14,0,10.80,ML,1604007233
2,2,Rikitea,-23.12,-134.97,74.95,76,61,12.01,PF,1604007198
3,3,Samarinda,-0.50,117.15,75.31,89,100,1.12,ID,1604007363
4,4,Ulaangom,49.98,92.07,25.05,86,6,2.15,MN,1604007593
...,...,...,...,...,...,...,...,...,...,...
574,574,Chumikan,54.72,135.31,30.27,93,72,11.83,RU,1604007202
575,575,Kirs,59.34,52.24,31.23,97,43,8.16,RU,1604007644
576,576,Tutóia,-2.76,-42.27,84.20,66,0,21.92,BR,1604007645
577,577,Meulaboh,4.14,96.13,75.74,84,87,4.05,ID,1604007243


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]


In [21]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Narrow down the cities to find ideal weather conditions by dropping cities with 
# max temp higher than 70 but lower than 80
# wind speed less than 10 mph,and zero cloudiness  

narrow_city_df = weather_data.loc[(weather_data["Max Temp"] <= 80) & (weather_data["Max Temp"] >= 70) & 
                                  (weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0)].dropna()

narrow_city_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
61,61,Hervey Bay,-25.30,152.85,77.00,72,0,4.70,AU,1604007205
87,87,Marsá Maţrūḩ,31.35,27.25,71.60,56,0,6.93,EG,1604007600
91,91,Salalah,17.02,54.09,77.00,69,0,5.82,OM,1604007219
111,111,Ormara,25.21,64.64,76.01,39,0,7.76,PK,1604007602
142,142,Vallenar,-28.57,-70.76,77.22,30,0,7.94,CL,1604006940
168,168,Ashquelon,31.67,34.57,75.00,100,0,5.93,IL,1604007609
258,258,Neuquén,-38.95,-68.06,73.40,18,0,6.93,AR,1604007616
288,288,Ghātanji,20.13,78.32,70.72,60,0,3.98,IN,1604007619
337,337,Kiryat Gat,31.61,34.76,75.00,100,0,1.01,IL,1604007245
379,379,Gaza,31.50,34.47,75.00,100,0,4.00,PS,1604007459


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [64]:
# Create the variable hotel_df
hotel_df = narrow_city_df.loc[:, ("Country", "City", "Lat", "Lng")]

# Add Hotel Name column to the DataFrame
hotel_df["Hotel Name"] = ""

# Display Result
hotel_df

,Country,City,Lat,Lng,Hotel Name
61,AU,Hervey Bay,-25.30,152.85,
87,EG,Marsá Maţrūḩ,31.35,27.25,
91,OM,Salalah,17.02,54.09,
111,PK,Ormara,25.21,64.64,
142,CL,Vallenar,-28.57,-70.76,
168,IL,Ashquelon,31.67,34.57,
258,AR,Neuquén,-38.95,-68.06,
288,IN,Ghātanji,20.13,78.32,
337,IL,Kiryat Gat,31.61,34.76,
379,PS,Gaza,31.50,34.47,


In [65]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
params = {"type": "lodging",
         "keyword": "hotel",
         "location": f"{lat},{lng}",
         "Radius": "5000",
         "key": f"{g_key}"}




In [66]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
request_url = base_url + "location=" + "30.48,-115.95" + "&radius=5000" + "&type=lodging" + "&key=" + g_key
requests.get(request_url).json()['results'][0]['name']



'Old Mill Hotel'

In [67]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = str(row["Lat"])
    lng = str(row["Lng"])
    city_name = row["City"]
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {city_name}.")
    
    request_url = base_url + "location=" + lat+"," + lng + "&radius=5000" + "&type=lodging" + "&key=" + g_key
    response = requests.get(request_url).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")


Closest hotel in Hervey Bay is Shelly Bay Resort.
------------
Closest hotel in Marsá Maţrūḩ is Beau Site.
------------
Closest hotel in Salalah is HAMDAN PLAZA HOTEL.
------------
Missing field/result... skipping.
------------
Closest hotel in Vallenar is Humacao Bed & Breakfast.
------------
Closest hotel in Ashquelon is Leonardo Ashkelon.
------------
Closest hotel in Neuquén is Hotel del Comahue.
------------
Closest hotel in Ghātanji is गजानन माहराज मदील घाटजी.
------------
Closest hotel in Kiryat Gat is OlusHome.
------------
Closest hotel in Gaza is رابطة الفنانين الفلسطينيين.
------------
Closest hotel in Dwārka is Hotel Narayan Avezika Comfort.
------------
Closest hotel in Maryborough is Blue Shades Motel.
------------
Closest hotel in Ambilobe is Hôtel & Restaurant DIANA Ambilobe.
------------
Closest hotel in Dzaoudzi is Le Rocher.
------------
Closest hotel in Bundaberg is Takalvan Motel.
------------
Closest hotel in Victoria Point is Bay Retreat Motel.
------------
Close

In [57]:
response

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}

In [58]:
hotel_df

,Country,City,Lat,Lng,Hotel Name
61,AU,Hervey Bay,-25.30,152.85,Shelly Bay Resort
87,EG,Marsá Maţrūḩ,31.35,27.25,Beau Site
91,OM,Salalah,17.02,54.09,HAMDAN PLAZA HOTEL
111,PK,Ormara,25.21,64.64,
142,CL,Vallenar,-28.57,-70.76,Humacao Bed & Breakfast
168,IL,Ashquelon,31.67,34.57,Leonardo Ashkelon
258,AR,Neuquén,-38.95,-68.06,Hotel del Comahue
288,IN,Ghātanji,20.13,78.32,गजानन माहराज मदील घाटजी
337,IL,Kiryat Gat,31.61,34.76,OlusHome
379,PS,Gaza,31.50,34.47,رابطة الفنانين الفلسطينيين


In [59]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [60]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))